<a href="https://colab.research.google.com/github/SEOUL-ABSS/SHIPSHIP/blob/main/SONAR6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==============================================================================
#                 DeepShip/MBARI 수중 음향 분류 프로젝트 (Data Leakage 해결 최종본)
# ==============================================================================
# 이 프로젝트는 YAMNet 모델을 이용하여 수중 음향 데이터를 분류하는 안정적이고 재현 가능한
# 베이스라인 파이프라인을 구축합니다. 최종 전문가 검토 의견을 반영하여 치명적 오류,
# 데이터 누수, 재현성 문제를 해결하고, 평가 파트를 강화하여 심층적인 분석을 수행합니다.
#
# [주요 개선 사항]
# 1. 데이터 유출 방지: GroupShuffleSplit을 사용하여 세그먼트가 아닌 파일 단위로 데이터를 분할.
#                      이를 통해 동일 파일의 세그먼트가 훈련/테스트셋에 섞이는 것을 원천 차단.
# 2. 안정성 및 재현성: 이전 검토 의견을 모두 반영하여 안정적인 파이프라인 유지.
# ==============================================================================


# ==============================================================================
# ## 1. 환경 설정 및 라이브러리 임포트
# ==============================================================================
print("1. 환경 설정 및 라이브러리 임포트 중...")

# --- 라이브러리 설치 ---
!pip install -q tensorflow tensorflow_hub soundfile librosa boto3 noisereduce umap-learn

# --- 모든 라이브러리 임포트 ---
import os, sys, subprocess, random, tempfile, shutil, gc, math
from collections import defaultdict
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import librosa, librosa.display, soundfile as sf
import boto3
from botocore import UNSIGNED
from botocore.client import Config
from sklearn.model_selection import train_test_split, GroupShuffleSplit # << GroupShuffleSplit 추가
from sklearn.preprocessing import LabelEncoder
import umap.umap_ as umap
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
# 평가지표 확장을 위해 추가
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
import noisereduce as nr

# --- 전역 시드 설정 (재현성 확보) ---
SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# --- Matplotlib 한글 폰트 설정 ---
!sudo apt-get -y install fonts-nanum > /dev/null
!sudo fc-cache -fv > /dev/null
import matplotlib.font_manager as fm
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
if os.path.exists(font_path):
    fm.fontManager.addfont(font_path); plt.rc('font', family='NanumGothic'); plt.rcParams['axes.unicode_minus'] = False
    print("\nMatplotlib 폰트 설정 완료: NanumGothic")
else:
    print("\n경고: 나눔고딕 폰트를 찾을 수 없습니다.")

# --- 전역 상수 정의 ---
print("\n전역 상수 정의 중...")
YAMNET_SAMPLE_RATE = 16000
DEEPSHIP_BASE_PATH = '/content/DeepShip'
MBARI_NOISE_BASE_DIR = '/content/MBARI_noise_data'
MODELS_TO_PROCESS = ['YAMNet']
print("전역 상수 정의 완료.")


# ==============================================================================
# ## 2. 데이터 확보
# ==============================================================================
print("\n2. 데이터 확보...")
if not os.path.exists(DEEPSHIP_BASE_PATH):
    try:
        subprocess.run(['git', 'clone', '--depth', '1', 'https://github.com/irfankamboh/DeepShip.git', DEEPSHIP_BASE_PATH], check=True, capture_output=True)
        print("DeepShip 클론 완료.")
    except Exception as e: print(f"오류: DeepShip 클론 실패: {e}")
else: print(f"DeepShip이 이미 존재합니다.")

os.makedirs(MBARI_NOISE_BASE_DIR, exist_ok=True)
if os.listdir(MBARI_NOISE_BASE_DIR): print(f"MBARI 노이즈 데이터가 이미 존재합니다.")
else:
    print(f"MBARI 노이즈 데이터 다운로드 시도 중...")
    try:
        s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))
        pages = s3.get_paginator('list_objects_v2').paginate(Bucket='pacific-sound-16khz', Prefix='2018/01/')
        dl_count = 0; MAX_DL = 10
        for page in pages:
            for obj in page.get('Contents', []):
                if obj['Key'].endswith('.wav') and obj.get('Size', 0) > 0:
                    local_path = os.path.join(MBARI_NOISE_BASE_DIR, os.path.basename(obj['Key']))
                    if not os.path.exists(local_path):
                        s3.download_file('pacific-sound-16khz', obj['Key'], local_path)
                        dl_count += 1
                if dl_count >= MAX_DL: break
            if dl_count >= MAX_DL: break
        print(f"MBARI 다운로드 완료. (파일 수: {dl_count})")
    except Exception as e: print(f"오류: MBARI 노이즈 데이터 다운로드 실패: {e}")
print("2. 데이터 확보 단계 완료.")


# ==============================================================================
# ## 3. 데이터 처리 및 세그먼테이션 함수 정의
# ==============================================================================
def load_and_segment_data(ship_path, noise_path, segment_duration=5.0, segment_overlap=0.5, undersample=True):
    hop_length = segment_duration * (1 - segment_overlap)
    all_data = []

    def _process_directory(base_path, label):
        segments = []
        print(f"'{label}' 클래스 데이터 처리 중: {base_path}")
        if not os.path.exists(base_path): return segments
        for root, _, files in os.walk(base_path):
            for file_name in sorted(files):
                if file_name.endswith('.wav'):
                    file_path = os.path.join(root, file_name)
                    try:
                        info = sf.info(file_path)
                        duration = info.duration
                        if duration < segment_duration: continue
                        for start_time in np.arange(0, duration - segment_duration + 1, hop_length):
                            segments.append(((file_path, start_time, info.samplerate), label))
                    except Exception as e: continue
        print(f"  '{label}' 클래스에서 총 {len(segments)}개의 세그먼트 생성 완료.")
        return segments

    ship_data = _process_directory(ship_path, 'ship')
    noise_data = _process_directory(noise_path, 'noise')
    all_data = ship_data + noise_data

    if undersample and ship_data and noise_data:
        n_ship = len(ship_data)
        n_noise = len(noise_data)
        if n_ship != n_noise:
            min_samples = min(n_ship, n_noise)
            print(f"\n클래스 불균형(Ship:{n_ship}, Noise:{n_noise}) -> 언더샘플링({min_samples}개) 수행.")
            ship_samples = random.sample(ship_data, min_samples)
            noise_samples = random.sample(noise_data, min_samples)
            all_data = ship_samples + noise_samples
            random.shuffle(all_data)

    if not all_data: print("\n오류: 처리할 세그먼트가 없습니다."); return [], [], False
    all_segments_info, all_labels = zip(*all_data)
    print("\n데이터 로드 및 세그먼테이션 완료.")
    return list(all_segments_info), list(all_labels), True


# ==============================================================================
# ## 4. 전처리 데이터셋 품질 평가 함수 정의
# ==============================================================================
def visualize_embeddings_with_umap(embeddings, labels, class_names):
    print("\n[데이터 품질 평가] UMAP 임베딩 시각화 실행 중...")
    if len(embeddings) < 3:
        print("  UMAP: 표본이 3개 미만이어서 시각화를 생략합니다.")
        return
    n_neighbors = max(2, min(15, len(embeddings) - 1))
    reducer = umap.UMAP(n_neighbors=n_neighbors, min_dist=0.1, n_components=2, random_state=SEED)
    embeddings_2d = reducer.fit_transform(embeddings)
    df = pd.DataFrame(embeddings_2d, columns=['x', 'y']); df['label'] = [class_names[l] for l in labels]
    plt.figure(figsize=(10, 8)); sns.scatterplot(data=df, x='x', y='y', hue='label', style='label', s=50, alpha=0.7)
    plt.title('UMAP을 이용한 임베딩 분포 시각화 (학습 전 진단)'); plt.grid(True); plt.show()

def visualize_spectrogram_samples(segments_info, labels, class_names, segment_duration, num_samples=3):
    print("\n[데이터 품질 평가] 스펙트로그램 샘플 시각화 실행 중...")
    unique_labels = np.unique(labels)
    fig, axes = plt.subplots(len(unique_labels), num_samples, figsize=(5*num_samples, 4*len(unique_labels)), squeeze=False)
    fig.suptitle('클래스별 스펙트로그램 샘플', fontsize=16)
    for i, label in enumerate(unique_labels):
        indices = np.where(labels == label)[0]
        if not len(indices): continue
        sample_indices = random.sample(list(indices), min(num_samples, len(indices)))
        for j, idx in enumerate(sample_indices):
            file_path, start_time, _ = segments_info[idx]
            ax = axes[i, j]
            try:
                y, _ = librosa.load(file_path, sr=YAMNET_SAMPLE_RATE, offset=start_time, duration=segment_duration)
                D = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)
                librosa.display.specshow(D, sr=YAMNET_SAMPLE_RATE, x_axis='time', y_axis='log', ax=ax)
                ax.set_title(f"{class_names[label]} Sample {j+1}")
            except Exception as e: ax.set_title(f"오디오 로드 실패")
    plt.tight_layout(rect=[0, 0.03, 1, 0.95]); plt.show()


# ==============================================================================
# ## 5. 오디오 전처리 및 임베딩 추출 함수 정의
# ==============================================================================
def mix_at_snr(clean, noise, snr_db):
    L = min(len(clean), len(noise))
    c, n = clean[:L].astype(np.float32), noise[:L].astype(np.float32)
    c_power = np.sqrt(np.mean(c**2)); n_power = np.sqrt(np.mean(n**2))
    if n_power < 1e-8: return c
    alpha = c_power / (n_power * (10**(snr_db / 20)))
    return c + alpha * n

def load_and_process_segment_efficient(file_info, duration, target_sr, config):
    file_path, start_time, orig_sr = file_info
    try:
        start_frame = int(start_time * orig_sr)
        num_frames = int(duration * orig_sr)
        y_segment, _ = sf.read(file_path, start=start_frame, stop=start_frame + num_frames, dtype='float32', always_2d=False)
        if y_segment.ndim > 1: y_segment = np.mean(y_segment, axis=1)
        if orig_sr != target_sr: y_segment = librosa.resample(y=y_segment, orig_sr=orig_sr, target_sr=target_sr)
        if config.get("apply_noise_reduction", False): y_segment = nr.reduce_noise(y=y_segment, sr=target_sr)
        if config.get("apply_rms_norm", False):
            rms = np.sqrt(np.mean(y_segment**2))
            if rms > 1e-6: y_segment = y_segment * (10.0**(-20.0/20.0)/rms)
        return y_segment
    except Exception as e: return None

def extract_yamnet_embedding(audio_info, model, config, noise_audio_infos=None, augment=False):
    try:
        y_segment = load_and_process_segment_efficient(audio_info, config["segment_duration"], YAMNET_SAMPLE_RATE, config)
        if y_segment is None: return None
        if augment and noise_audio_infos:
            noise_info = random.choice(noise_audio_infos)
            y_noise = load_and_process_segment_efficient(noise_info, config["segment_duration"], YAMNET_SAMPLE_RATE, {"apply_noise_reduction":False, "apply_rms_norm":True})
            if y_noise is not None:
                if config.get("use_snr_augmentation", False):
                    snr_db = random.uniform(config["snr_min_db"], config["snr_max_db"])
                    y_segment = mix_at_snr(y_segment, y_noise, snr_db)
                else:
                    min_len = min(len(y_segment), len(y_noise))
                    y_segment = y_segment[:min_len] + y_noise[:min_len] * config["noise_level"]
        _, embeddings, _ = model(y_segment)
        return tf.reduce_mean(embeddings, axis=0).numpy() if embeddings.shape[0] > 0 else None
    except Exception as e: return None


# ==============================================================================
# ## 6. 모델 관련 함수 정의 (평가 파트 강화)
# ==============================================================================
def load_audio_models():
    models = {}; print("\n오디오 모델 로드 중...")
    try:
        models['YAMNet'] = hub.load('https://tfhub.dev/google/yamnet/1')
        print("  YAMNet 모델 로드: 성공"); return models, True
    except Exception as e:
        print(f"  모델 로드 실패: {e}"); return {}, False

def build_classifier_model(input_shape, num_classes, learning_rate):
    inp = Input(shape=(input_shape,), name='embedding_input')
    x = Dense(256, activation='relu')(inp); x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x); x = Dropout(0.5)(x)
    out = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=inp, outputs=out)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def analyze_top_errors(y_true, y_pred_probs, segments_info, class_names, top_k=5):
    """모델이 가장 헷갈려하는 Top-K 오류 샘플을 분석하고 스펙트로그램을 시각화합니다."""
    print(f"\n--- Top-{top_k} 오류 심층 분석 ---")
    errors = []
    ship_idx = list(class_names).index('ship')

    for i in range(len(y_true)):
        true_label = y_true[i]
        pred_label = np.argmax(y_pred_probs[i])

        if true_label != pred_label:
            error_magnitude = 0; error_type = ""
            if pred_label == ship_idx: # False Positive (Noise -> Ship)
                error_type = "FP (Noise->Ship)"; error_magnitude = y_pred_probs[i][ship_idx]
            else: # False Negative (Ship -> Noise)
                error_type = "FN (Ship->Noise)"; error_magnitude = 1 - y_pred_probs[i][ship_idx]
            errors.append({
                "index": i, "type": error_type, "magnitude": error_magnitude, "info": segments_info[i],
                "true_label": class_names[true_label], "pred_label": class_names[pred_label],
                "confidence": y_pred_probs[i][pred_label]
            })

    sorted_errors = sorted(errors, key=lambda x: x['magnitude'], reverse=True)
    if not sorted_errors: print("  오류가 발견되지 않았습니다."); return

    print(f"가장 확신하며 틀린 {min(top_k, len(sorted_errors))}개 샘플:")
    for error in sorted_errors[:top_k]:
        print(f"  - 타입: {error['type']}, 신뢰도: {error['confidence']:.2f}, 실제: {error['true_label']}, 예측: {error['pred_label']}")
        print(f"    - 파일: {os.path.basename(error['info'][0])} (시작: {error['info'][1]:.2f}s)")

    fig, axes = plt.subplots(1, min(top_k, len(sorted_errors)), figsize=(5*min(top_k, len(sorted_errors)), 4), squeeze=False)
    fig.suptitle('Top-K 오류 샘플 스펙트로그램', fontsize=16)
    for i, error in enumerate(sorted_errors[:top_k]):
        ax = axes[0, i]
        file_path, start_time, _ = error['info']
        try:
            y, _ = librosa.load(file_path, sr=YAMNET_SAMPLE_RATE, offset=start_time, duration=5.0)
            D = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)
            librosa.display.specshow(D, sr=YAMNET_SAMPLE_RATE, x_axis='time', y_axis='log', ax=ax)
            ax.set_title(f"{error['type']}\nTrue:{error['true_label']}, Pred:{error['pred_label']} ({error['confidence']:.2f})")
        except Exception as e: ax.set_title("로드 실패")
    plt.tight_layout(rect=[0, 0.03, 1, 0.93]); plt.show()

def evaluate_and_visualize_model(model_name, trained_model, history, test_ds, test_steps, y_true_filtered, test_segments_info, label_encoder):
    print(f"\n--- {model_name} 모델 성능 평가 및 시각화 ---")
    loss, accuracy = trained_model.evaluate(test_ds, steps=test_steps, verbose=0)
    y_pred_probs = trained_model.predict(test_ds, steps=test_steps)
    if len(y_pred_probs) != len(y_true_filtered): y_pred_probs = y_pred_probs[:len(y_true_filtered)]
    y_pred = np.argmax(y_pred_probs, axis=1)

    report_dict = classification_report(y_true_filtered, y_pred, target_names=label_encoder.classes_, output_dict=True)
    f1_macro = report_dict['macro avg']['f1-score']

    ship_idx = list(label_encoder.classes_).index('ship')
    y_scores = y_pred_probs[:, ship_idx]
    fpr, tpr, _ = roc_curve(y_true_filtered, y_scores, pos_label=ship_idx)
    auc_score = auc(fpr, tpr)

    print(f"  테스트 손실: {loss:.4f}, 정확도: {accuracy:.4f}, Macro F1: {f1_macro:.4f}, AUC: {auc_score:.4f}")

    print("\n  [분류 리포트]"); print(classification_report(y_true_filtered, y_pred, target_names=label_encoder.classes_))
    cm = confusion_matrix(y_true_filtered, y_pred)
    plt.figure(figsize=(6, 5)); sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
    plt.xlabel('예측 레이블'); plt.ylabel('실제 레이블'); plt.title(f'{model_name} 혼동 행렬'); plt.show()

    plt.figure(figsize=(7, 6)); plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {auc_score:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--'); plt.xlim([0.0, 1.0]); plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate'); plt.title('Receiver Operating Characteristic (ROC) Curve'); plt.legend(loc="lower right"); plt.grid(True); plt.show()

    plt.figure(figsize=(12, 5)); plt.subplot(1, 2, 1); plt.plot(history.history['accuracy'], label='훈련 정확도'); plt.plot(history.history['val_accuracy'], label='검증 정확도')
    plt.title('훈련 및 검증 정확도'); plt.legend(); plt.subplot(1, 2, 2); plt.plot(history.history['loss'], label='훈련 손실'); plt.plot(history.history['val_loss'], label='검증 손실')
    plt.title('훈련 및 검증 손실'); plt.legend(); plt.show()

    analyze_top_errors(y_true_filtered, y_pred_probs, test_segments_info, label_encoder.classes_)

    return {'loss': loss, 'accuracy': accuracy, 'f1_macro': f1_macro, 'auc': auc_score}


# ==============================================================================
# ## 7. 전체 파이프라인 실행 및 결과 비교
# ==============================================================================
CONFIG = {
    "segment_duration": 5.0, "segment_overlap": 0.5, "undersample": True,
    "apply_noise_reduction": False, "apply_rms_norm": True,
    "use_snr_augmentation": True, "snr_min_db": 0, "snr_max_db": 15, "noise_level": 0.05,
    "run_quality_evaluation": True, "quality_eval_samples": 300,
    "test_size": 0.2, "epochs": 50, "batch_size": 16, "learning_rate": 0.0005,
}

print(">>> 1. 데이터 로드 및 세그먼테이션 시작...")
all_segments_info, all_labels, is_data_ready = load_and_segment_data(DEEPSHIP_BASE_PATH, MBARI_NOISE_BASE_DIR, **{k:v for k,v in CONFIG.items() if k in ['segment_duration', 'segment_overlap', 'undersample']})

if is_data_ready:
    print("\n>>> 2. Group-based 데이터 분할 시작...")
    label_encoder = LabelEncoder(); encoded_labels = label_encoder.fit_transform(all_labels)
    num_classes = len(label_encoder.classes_)
    groups = [info[0] for info in all_segments_info]
    gss = GroupShuffleSplit(n_splits=1, test_size=CONFIG["test_size"], random_state=SEED)
    train_idx, test_idx = next(gss.split(all_segments_info, encoded_labels, groups))
    X_train_info = [all_segments_info[i] for i in train_idx]; y_train_enc = encoded_labels[train_idx]
    X_test_info = [all_segments_info[i] for i in test_idx]; y_test_enc = encoded_labels[test_idx]
    print(f"데이터 분할 완료 (파일 단위): 훈련 {len(X_train_info)}개, 테스트 {len(X_test_info)}개")
    train_files_set = set(info[0] for info in X_train_info); test_files_set = set(info[0] for info in X_test_info)
    print(f"  훈련셋 고유 파일 수: {len(train_files_set)}"); print(f"  테스트셋 고유 파일 수: {len(test_files_set)}"); print(f"  두 세트 간 중복 파일 수: {len(train_files_set.intersection(test_files_set))}")
else: is_data_ready = False

if is_data_ready:
    print("\n>>> 3. 오디오 모델 로드 시작..."); loaded_models, are_models_loaded = load_audio_models()
    if not are_models_loaded: is_data_ready = False

if is_data_ready and CONFIG["run_quality_evaluation"]:
    print("\n>>> 4. 데이터셋 품질 사전 평가 시작...")
    sample_indices = random.sample(range(len(X_train_info)), min(CONFIG["quality_eval_samples"], len(X_train_info)))
    sample_info = [X_train_info[i] for i in sample_indices]; sample_labels = y_train_enc[sample_indices]
    sample_embeddings = [extract_yamnet_embedding(info, loaded_models['YAMNet'], CONFIG) for info in sample_info]
    valid_embeddings = [emb for emb in sample_embeddings if emb is not None]; valid_labels = [label for emb, label in zip(sample_embeddings, sample_labels) if emb is not None]
    if valid_embeddings:
        visualize_embeddings_with_umap(np.array(valid_embeddings), valid_labels, label_encoder.classes_)
        visualize_spectrogram_samples(X_train_info, y_train_enc, label_encoder.classes_, CONFIG["segment_duration"])
    else: print("품질 평가용 임베딩 추출 실패.")

if is_data_ready:
    print("\n>>> 5. 모델별 학습, 평가 및 결과 비교 시작...")
    evaluation_results = {}; temp_dir = tempfile.mkdtemp()
    noise_files_train = [info for info, lbl in zip(X_train_info, y_train_enc) if label_encoder.inverse_transform([lbl])[0] == 'noise']
    if not noise_files_train: print("경고: 훈련 데이터에 노이즈 샘플이 없어 증강을 수행할 수 없습니다.")

    def make_dataset(file_paths, batch_size, num_classes, input_dim):
        def gen():
            while True:
                random.shuffle(file_paths)
                for i in range(0, len(file_paths), batch_size):
                    paths = file_paths[i:i+batch_size]; embs, labs = [], []
                    for p in paths:
                        try:
                            with np.load(p) as d: embs.append(d['embedding']); labs.append(d['label'])
                        except Exception as e: continue
                    if not embs: continue
                    yield (np.asarray(embs, dtype=np.float32), tf.keras.utils.to_categorical(np.array(labs), num_classes=num_classes))
        output_signature = (tf.TensorSpec(shape=(None, input_dim), dtype=tf.float32), tf.TensorSpec(shape=(None, num_classes), dtype=tf.float32))
        return tf.data.Dataset.from_generator(gen, output_signature=output_signature).prefetch(tf.data.AUTOTUNE)

    for model_name in MODELS_TO_PROCESS:
        print(f"\n{'='*25} {model_name} 모델 처리 시작 {'='*25}")
        model_hub = loaded_models[model_name]; train_files, test_files, y_train_filt, y_test_filt = [], [], [], []

        for split, info_list, y_list, out_files, out_y in [('train', X_train_info, y_train_enc, train_files, y_train_filt), ('test', X_test_info, y_test_enc, test_files, y_test_filt)]:
            print(f"  {model_name}: {split} 데이터 임베딩 추출 중...")
            for i, (segment_info, label) in enumerate(zip(info_list, y_list)):
                is_ship = (label_encoder.inverse_transform([label])[0] == 'ship')
                noise_src = noise_files_train if split == 'train' else None
                emb = extract_yamnet_embedding(segment_info, model_hub, CONFIG, noise_audio_infos=noise_src, augment=(split=='train' and is_ship))
                if emb is not None:
                    path = os.path.join(temp_dir, f'{model_name}_{split}_{i}.npz')
                    np.savez_compressed(path, embedding=emb, label=label)
                    out_files.append(path); out_y.append(label)

        if len(train_files) < CONFIG["batch_size"] or not test_files:
            print(f"  {model_name}: 데이터 부족으로 학습/평가를 건너뜁니다."); continue

        try:
            with np.load(train_files[0]) as data: input_dim = data['embedding'].shape[-1]
            print(f"  임베딩 차원을 동적으로 확인: {input_dim}")
        except Exception as e:
            print(f"  임베딩 차원 확인 실패. 기본값을 사용합니다."); input_dim = YAMNET_EMBEDDING_DIM

        classifier = build_classifier_model(input_dim, num_classes, CONFIG["learning_rate"])
        train_ds = make_dataset(train_files, CONFIG["batch_size"], num_classes, input_dim)
        test_ds  = make_dataset(test_files,  CONFIG["batch_size"], num_classes, input_dim)
        train_steps = math.ceil(len(train_files) / CONFIG["batch_size"]); test_steps = math.ceil(len(test_files) / CONFIG["batch_size"])

        history = classifier.fit(train_ds,
            steps_per_epoch=train_steps, validation_data=test_ds, validation_steps=test_steps,
            epochs=CONFIG["epochs"],
            callbacks=[EarlyStopping(patience=10, restore_best_weights=True), ReduceLROnPlateau(patience=5)], verbose=1
        )

        results = evaluate_and_visualize_model(model_name, classifier, history, test_ds, test_steps, np.array(y_test_filt), X_test_info, label_encoder)
        evaluation_results[model_name] = results; gc.collect()

    shutil.rmtree(temp_dir)
    print("\n--- 6. 최종 성능 비교 (F1, AUC 포함) ---")
    if evaluation_results:
        results_df = pd.DataFrame(evaluation_results).T.sort_values('auc', ascending=False)
        print(results_df.to_string(formatters={
            'loss': '{:.4f}'.format, 'accuracy': '{:.4f}'.format,
            'f1_macro': '{:.4f}'.format, 'auc': '{:.4f}'.format
        }))
    else: print("평가된 모델이 없습니다.")

print("\n🎉 전체 파이프라인 실행 완료.")